### 인터파크 - 베스트셀러

In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup


- Step1) 사이트에 접속하고 HTML 데이터를 가져와 파싱

In [2]:
url = 'http://book.interpark.com/display/collectlist.do?_method=bestsellerHourNew&bookblockname=b_gnb&booklinkname=%BA%A3%BD%BA%C6%AE%C1%B8&bid1=bgnb_mn&bid2=LiveRanking&bid3=main&bid4=001'
res = requests.get(url)     # url 가져오기
soup = BeautifulSoup(res.text, 'html.parser')

- Step 2) 원하는 데이터 찾기

In [5]:
lis = soup.select('.rankBestContentList > ol > li')
len(lis)

15

- Step 3) 하나를 선택해서 원하는 정보 추출

In [139]:
li = lis[14]
li

<li>
<!-- listItem | singleType or setType :: start -->
<div class="listItem singleType">
<div class="cover">
<div class="coverImage">
<label>
<a href="/product/BookDisplay.do?_method=detail&amp;sc.shopNo=0000400000&amp;sc.prdNo=356754244&amp;sc.saNo=003003001&amp;bid1=Best_zone&amp;bid2=LiveRanking&amp;bid3=PRD&amp;bid4=001" target="_blank">
<span class="checkBox">
<input name="_KEY" type="checkbox" value="356754244"/>
</span>
<img alt="비가 오면 열리는 상점" src="https://qi-b.qoo10cdn.com/partner/goods_image/4/2/4/4/356754244s.jpg"/>
<i class="decoFront"></i>
</a>
</label>
</div>
<div class="coverOption">
<ul>
<li class="cart">
<a class="cartGo" href="#" value="356754244"><span>CART</span></a>
</li>
<li class="buy">
<a href="javascript:directOrd('356754244');"><span>BUY</span></a>
</li>
</ul>
</div>
<div class="rankNumber digit2">
<span class="rankBtn_ctrl rkNum_M01"></span><span class="rankBtn_ctrl rkNum_M02"></span>
</div>
</div>
<div class="itemIcon">
<span class="bestSeller">베스트셀러</span>


In [154]:
# 랭크
rank = li.select_one('.cover').find('div', class_='rankNumber digit2').find_all('span')
rank

[<span class="rankBtn_ctrl rkNum_M01"></span>,
 <span class="rankBtn_ctrl rkNum_M02"></span>]

In [137]:
rank = li.select_one('.cover').find('div', class_='rankNumber digit2').find_all('span')
if len(rank) == 2:
    s = ''
    r1 = str(rank[0])
    r1 = r1[-10]
    s += r1
    r2 = str(rank[1])
    r2 = r2[-10]
    s += r2
else:
    s = ''
    
    s+=str(rank[0])[-10]
    
s

'12'

In [14]:
# 책 제목
title = li.select_one('.itemName').get_text().strip()
title

'도시와 그 불확실한 벽'

In [12]:
# 저자
author = li.select_one('.author').get_text().strip()
author

'무라카미 하루키(村上春樹)'

In [17]:
# 출판사
company = li.select_one('.company').get_text().strip()
company

'문학동네'

In [18]:
# 가격
price = li.select_one('.price > em').get_text().strip()
price = int(price.replace(',', ''))
price

17550

- Step 4) 반복문을 사용하여 페이지 내에 있는 데이터 가져오기

In [151]:
data = []
for li in lis:
    title = li.select_one('.itemName').get_text().strip()
    author = li.select_one('.author').get_text().strip()
    company = li.select_one('.company').get_text().strip()
    price = li.select_one('.price > em').get_text().strip()
    price = int(price.replace(',', ''))
    rank = li.select_one('.cover').find('div', class_='rankNumber digit2').find_all('span')
    if len(rank) == 2:
        r1, r2 = str(rank[0]), str(rank[1])
        r1, r2 = r1[-10], r2[-10]
        s = r1+r2
    else:
        s = ''
        s+=str(rank[0])[-10]
        s
    data.append({'제목': title, '저자':author, '출판사':company, '가격':price, '순위':s})

- Step 5) 오류가 발생하면 오류를 해결하고 2, 3, 4 과정을 반복

- Step 6) 모든 페이지에 대해 반복하기

In [152]:
df = pd.DataFrame(data)
df

,제목,저자,출판사,가격,순위
0,도시와 그 불확실한 벽,무라카미 하루키(村上春樹),문학동네,17550,1
1,흔한남매 14,"흔한남매, 백난도/유난희 그림/흔한컴퍼니 감수",미래엔아이세움,13050,2
2,일론 머스크,월터 아이작슨,21세기북스,34200,3
3,퓨처 셀프,벤저민 하디/최은아 역,상상스퀘어,17820,4
4,가짜 노동,"데니스 뇌르마르크, 아네르스 포그 옌센/이수영 역",자음과모음,15120,5
5,어떤 하락장에서도 살아남을 수 있는 시장 불변의 법칙,장지웅,여의도책방,18000,6
6,디케의 눈물,조국,다산북스,16920,7
7,트렌드 코리아 2024,"김난도, 추예린, 전다현, 전미영, 최지혜",미래의창,17100,8
8,1%를 읽는 힘,메르,토네이도,19800,9
9,세이노의 가르침,세이노,데이원,6480,9
